In [1]:
import os
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import torch.optim as optim
torch.autograd.set_detect_anomaly(True)
cfgfile = "C:/Users/HX/Desktop/yolov4.cfg"
model_file_path = "Model/model.pt"

def read_cfg_file(cfgfile):
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')

    layer_type = []
    layer_details = []
    current_layer_details = {}
    for line in lines:
        #print(line)
        if line == '':
            continue
        elif line[0] == '#':
            continue
        else:
            if (line[0] == '['):
                layer_type.append(line[1 : -1])
                if current_layer_details != {}:
                    layer_details.append(current_layer_details)
                    current_layer_details = {}
            else:
                current_layer_details.update([(line.split("=")[0].rstrip(), line.split("=")[1].lstrip())])
    layer_details.append(current_layer_details)
    return layer_type, layer_details

 

In [2]:
class Mish(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * torch.tanh(F.softplus(x))
        return x

class Conv_Layer_box(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, activation_func, batch_normalization):
        super().__init__()
        padding = (int((kernel_size - 1)/2), int((kernel_size - 1)/2))
        #TBC: linear
        dict_activation_func = {"ReLU": nn.ReLU(inplace=False),
                                "linear": nn.ReLU(inplace=False),
                                "leaky": nn.LeakyReLU(0.1, inplace=False),
                                "mish": Mish()
                               }
        
        if batch_normalization == True:
            bias = False
        else:
            bias = True
        self.conv_box = nn.ModuleList()
        self.conv_box.append(nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, bias = bias))
        if batch_normalization == True:
            self.conv_box.append(nn.BatchNorm2d(out_channel))
        self.conv_box.append(dict_activation_func[activation_func])
        
    def forward(self, x):
        for layer in self.conv_box:
            x = layer(x)
        return x
    
class Maxpool_pad_Layer_box(nn.Module):
    def __init__(self, maxpool_size):
        super().__init__()
        self.maxpool_size = maxpool_size
        #why there are 2 padding??????????????
        self.pad_1 = int((self.maxpool_size - 1) / 2)
        self.pad_2 = self.pad_1
    def forward(self, x):
        x = F.pad(x, (self.pad_1, self.pad_2, self.pad_1, self.pad_2), mode='replicate')
        x = F.max_pool2d(x, self.maxpool_size, stride=1)
        return x
    
class Upsample_layer(nn.Module):
    def __init__(self, stride):
        super().__init__()
        self.stride = stride
        
    def forward(self, x):
        batch, channel, height, width = x.data.size()
        x = x.view(batch, channel, height, 1, width, 1).expand(batch, channel, height, self.stride, width, self.stride).clone()
        x = x.contiguous().view(batch, channel, height * self.stride, width * self.stride).clone()
        return x
    

        
        
class shortcut(nn.Module):
    def __init__(self):
        super().__init__()
        
class route(nn.Module):
    def __init__(self):
        super().__init__()

In [3]:
def cross_length(a_1, a_2, b_1, b_2):
    if a_1 <=b_1 and a_2 >= b_1:
        return (min(a_2, b_2) - b_1)
    elif a_1 <=b_1 and a_2 <= b_1:
        return 0
    else:
        return cross_length(b_1, b_2, a_1, a_2)

def IoU(x_GT, y_GT, h_GT, w_GT, x_PD, y_PD, h_PD, w_PD):
    area_of_I = cross_length(x_GT, x_GT + h_GT, x_PD, x_PD + h_PD) * cross_length(y_GT, y_GT + h_GT, y_PD, y_PD + h_PD)
    area_of_U = h_GT * w_GT + h_PD * w_PD - area_of_I
    return area_of_I / area_of_U



def axis_conversion(x_centre, y_centre, h, w):
    return (x_centre - h / 2, y_centre - w / 2, h, w)

In [4]:
25/(64)
axis_conversion(6, 6, 3,3)
#IoU(11,11,8,8,12,12,5,5)

(4.5, 4.5, 3, 3)

In [5]:
Yolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)
Yolo_input = torch.from_numpy(Yolo_input)
#input[:,:,0,0] = 2
#input[:,:,0,0]
anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
mask = [0, 1, 2]
classes = 80
input_image_size = 608

In [6]:
#add_on_Matrix_x = torch.from_numpy(np.array([[i for j in range(19)] for i in range(19)]))
#add_on_Matrix_y = [[i for i in range(19)] for j in range(19)]
#add_on_Matrix_x

In [7]:
class Yolo(nn.Module):
    def __init__(self, anchors, mask, classes, input_image_size):
        super().__init__()
        self.anchors = anchors
        self.mask = mask
        self.classes = classes
        self.number_of_mask = len(mask)
        self.input_image_size = input_image_size
        #self.Sigmoid_layer = nn.Sigmoid()

    def forward(self, x):
        
        mask = self.mask
        anchors = self.anchors
        classes = self.classes
        number_of_mask = self.number_of_mask
        input_image_size = self.input_image_size
        grid_size = int(input_image_size / x.size(2))
        #print(grid_size)
        #t_x = torch.from_numpy(np.array([0.0 for i in range(number_of_mask * x.size(0) * x.size(2) * x.size(3))]).reshape(number_of_mask * x.size(0), 1, x.size(2), x.size(3)))
        t_x = [None for i in range(number_of_mask)]
        t_y = [None for i in range(number_of_mask)]
        t_w = [None for i in range(number_of_mask)]
        t_h = [None for i in range(number_of_mask)]
        objective_p = [None for i in range(number_of_mask)]
        class_p = [None for i in range(number_of_mask)]
        
        #c_x = [i for i in range(x.size(2))]
        #c_y = [i for i in range(x.size(2))]
        
        add_on_Matrix_x = torch.from_numpy(np.array([[i for j in range(x.size(2))] for i in range(x.size(2))])).cuda()
        add_on_Matrix_y = torch.from_numpy(np.array([[i for i in range(x.size(2))] for j in range(x.size(2))])).cuda()
        
        b_x = [None for i in range(number_of_mask)]
        b_y = [None for i in range(number_of_mask)]
        b_w = [None for i in range(number_of_mask)]
        b_h = [None for i in range(number_of_mask)]
        
        
        anchor_shape_1 = int(len(anchors) / 2)
        anchors = np.array(anchors).reshape(anchor_shape_1, 2)
        
        print(anchors)
        
        p_w = [None for i in range(number_of_mask)]
        p_h = [None for i in range(number_of_mask)]
        
        
        for i in range(number_of_mask):
            start_point = i * (5 + classes)
            end_point = (i + 1) * (5 + classes)
            p_w[i], p_h[i] = anchors[mask[i]]
            print(p_w)
            print(p_h)
            
            t_x[i] = x[:, (start_point + 0) : (start_point + 1), :, :].clone()
            
            t_y[i] = x[:, (start_point + 1) : (start_point + 2), :, :].clone()
            t_w[i] = x[:, (start_point + 2) : (start_point + 3), :, :].clone()
            t_h[i] = x[:, (start_point + 3) : (start_point + 4), :, :].clone()
            objective_p[i] = x[:, (start_point + 4) : (start_point + 5), :, :].clone()
            class_p[i] = x[:, (start_point + 5) : end_point, :, :].clone()
            #print(type(t_x[i]))
            
            print(F.sigmoid(t_x[i].clone()))
            b_x[i] = F.sigmoid(t_x[i].clone()) + add_on_Matrix_x
            print(b_x[i])
            b_y[i] = F.sigmoid(t_y[i].clone()) + add_on_Matrix_y
            
            #b_x[i][0, 0, :, :] = b_x[i][0, 0, :, :].clone()
            #b_y[i][0, 0, :, :] = b_y[i][0, 0, :, :].clone()
            
            """
            print(t_x[i].size())
            for m in range(x.size(2)):
                for n in range(x.size(2)):
                    b_x[i][:, :, c_x[m], c_y[n]] = c_x[m] + b_x[i][:, :, c_x[m], c_y[n]].clone()
                    b_y[i][:, :, c_x[m], c_y[n]] = c_y[n] + b_y[i][:, :, c_x[m], c_y[n]].clone()
            """
            #need to think whether need to use below 2 lines
            b_x[i] = grid_size * b_x[i].clone()
            b_y[i] = grid_size * b_y[i].clone()
            b_w[i] = p_w[i] * torch.exp(t_w[i].clone())
            b_h[i] = p_h[i] * torch.exp(t_h[i].clone())
            
            objective_p[i] = F.sigmoid(objective_p[i].clone())
            class_p[i] = F.sigmoid(class_p[i].clone())
            #torch.reshape(t_x[i])
        
        b_x = torch.stack(b_x).clone()
        b_y = torch.stack(b_y).clone()
        b_w = torch.stack(b_w).clone()
        b_h = torch.stack(b_h).clone()
        objective_p = torch.stack(objective_p).clone()
        class_p = torch.stack(class_p).clone()
        combined_yolo_output = torch.cat((b_x, b_y, b_w, b_h, objective_p, class_p), 2)
        #return b_x, b_y, b_w, b_h, objective_p, class_p
        #return combined_yolo_output
        
        
        #b_x = torch.stack(b_x).clone()
        return combined_yolo_output


"""

#Yolo_run_layer = Yolo(anchors, mask, classes, input_image_size)

#b_x, b_y, b_w, b_h, objective_p, class_p = Yolo_run_layer(Yolo_input)
#combined_yolo_output = torch.cat((b_x, b_y, b_w, b_h, objective_p, class_p), 2)

Yolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)
Yolo_input = torch.from_numpy(Yolo_input)
target = np.array([0 for i in range(3 * 76 * 76 * 85)])
input_tensor = Yolo_input
output_tensor = torch.Tensor(target)

learning_rate = 0.08
epoch_size = 5
steps_for_printing_out_loss = 1

YOLO_Module_WIP = Yolo(anchors, mask, classes, input_image_size)
YOLO_Module_WIP.cuda()
#Model_WIP.to(device)
loss_functioin = nn.MSELoss()
optimizer = optim.SGD(YOLO_Module_WIP.parameters(), lr = learning_rate)

input = input_tensor.cuda()
target = output_tensor.cuda()

def training_model():
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output = YOLO_Module_WIP(input.cuda())
        print(output.size())
        #b_x, b_y, b_w, b_h, objective_p, class_p = YOLO_v4_Module_WIP(input.cuda())
        #output = b_x
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3), output.size(4)))
        loss.backward()
        optimizer.step()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
    torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)

training_model()
"""

"\n\n#Yolo_run_layer = Yolo(anchors, mask, classes, input_image_size)\n\n#b_x, b_y, b_w, b_h, objective_p, class_p = Yolo_run_layer(Yolo_input)\n#combined_yolo_output = torch.cat((b_x, b_y, b_w, b_h, objective_p, class_p), 2)\n\nYolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)\nYolo_input = torch.from_numpy(Yolo_input)\ntarget = np.array([0 for i in range(3 * 76 * 76 * 85)])\ninput_tensor = Yolo_input\noutput_tensor = torch.Tensor(target)\n\nlearning_rate = 0.08\nepoch_size = 5\nsteps_for_printing_out_loss = 1\n\nYOLO_Module_WIP = Yolo(anchors, mask, classes, input_image_size)\nYOLO_Module_WIP.cuda()\n#Model_WIP.to(device)\nloss_functioin = nn.MSELoss()\noptimizer = optim.SGD(YOLO_Module_WIP.parameters(), lr = learning_rate)\n\ninput = input_tensor.cuda()\ntarget = output_tensor.cuda()\n\ndef training_model():\n    for i in range(1, epoch_size + 1):\n        optimizer.zero_grad()\n        output = YOLO_Module_WIP(input.cuda())\n        print(output

In [8]:
#combined_yolo_output

In [9]:
layer_type, layer_details = read_cfg_file(cfgfile)

net_layer = layer_details[0]

layer_type = layer_type[1:]
layer_details = layer_details[1:]

print(len(layer_type))
print(len(layer_details))
print(layer_details[1])

162
162
{'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'mish'}


In [10]:
"""
anchor = 3
#yolo_layer = 3
output = (19 ** 2) * (1 + 4 + 16)
anchor * output
"""

'\nanchor = 3\n#yolo_layer = 3\noutput = (19 ** 2) * (1 + 4 + 16)\nanchor * output\n'

In [11]:
#build module for entire YOLO
class YOLO_v4_model(nn.Module):
    def __init__(self, layer_details, layer_type):
        super(YOLO_v4_model, self).__init__()
        self.all_layers = nn.ModuleList()
        all_layers = self.all_layers
        self.layer_details = layer_details
        self.layer_type = layer_type

        for i in range(len(layer_type)):
            if layer_type[i] == 'convolutional':
                #print(layer_details[i])
                #print(i)
                try:
                    if int(layer_details[i]['batch_normalize']) == 1:
                        batch_normalize = True
                    else:
                        batch_normalize = False
                except:
                    batch_normalize = False
                if i == 0:
                    in_channel = 3
                else:
                    if layer_type[i - 1] == 'convolutional':
                        skip_step = 0
                    elif layer_type[i - 1] == 'shortcut':
                        skip_step = int(layer_details[i - 1]['from'])
                    elif layer_type[i - 1] == 'route':
                        try:
                            skip_step = int(layer_details[i - 1]['layers'].split(",")[0])
                        except:
                            skip_step = int(layer_details[i - 1]['layers'])
                    #print(layer_details[i - 1 + skip_step])
                    if skip_step > 0:
                        in_channel = int(layer_details[skip_step]['filters'])
                    else:
                        in_channel = int(layer_details[i - 1 + skip_step]['filters'])
                out_channel = int(layer_details[i]['filters'])
                kernel_size = int(layer_details[i]['size'])
                stride = int(layer_details[i]['stride'])
                pad = int(layer_details[i]['pad'])
                activation_func = layer_details[i]['activation']
                layer = Conv_Layer_box(in_channel, out_channel, kernel_size, stride, activation_func, batch_normalize)
                #print(layer)
            elif layer_type[i] == 'maxpool':
                layer_details[i].update([('filters', layer_details[i - 1]['filters'])])
                maxpool_size = int(layer_details[i]['size'])
                #print(maxpool_size)
                layer = Maxpool_pad_Layer_box(maxpool_size)
                #print(layer)
            elif layer_type[i] == 'upsample':
                layer_details[i].update([('filters', layer_details[i - 1]['filters'])])
                stride = int(layer_details[i]['stride'])
                layer = Upsample_layer(stride)
            elif layer_type[i] == 'yolo':
                #print("yolo")
                anchors = [int(x) for x in layer_details[i]['anchors'].split(",")]
                
                mask = [int(x) for x in layer_details[i]['mask'].split(",")]
                classes = int(layer_details[i]['classes'])
                #input image size = 608 for now
                layer = Yolo(anchors, mask, classes, input_image_size)
                #print(anchors)
                #print(classes)
                #print(input_image_size)
                #continue
            elif layer_type[i] == 'shortcut':
                skip_step = int(layer_details[i]['from'])
                layer_details[i].update([('filters', layer_details[i + skip_step]['filters'])])
                layer = shortcut()
            elif layer_type[i] == 'route':
                try:
                    skip_step = int(layer_details[i]['layers'].split(",")[0])
                except:
                    skip_step = int(layer_details[i]['layers'])
                #print(skip_step)
                if skip_step > 0:
                    layer_details[i].update([('filters', layer_details[skip_step]['filters'])])
                else:
                    layer_details[i].update([('filters', layer_details[i + skip_step]['filters'])])
                layer = route()
            elif layer_type[i] == 'net':
                #print("net")
                continue
            else:
                continue

            all_layers.append(layer)
        global all_layerrr
        all_layerrr = all_layers

    def forward(self, x):
        all_layers = self.all_layers
        layers_output = [None for i in range(len(layer_type))]
        for i in range(len(layer_type)):
            #print(i)
            if i == 0:
                layers_output[i] = all_layers[i](x)
                continue
                
            elif layer_type[i] == 'yolo':
                layers_output[i] = all_layers[i](layers_output[i - 1])
                continue
            elif layer_type[i] == 'convolutional' or layer_type[i] == 'maxpool' or layer_type[i] == 'upsample' or layer_type[i] == 'yolo':
                layers_output[i] = all_layers[i](layers_output[i - 1])
                """
                try:
                    print("i: " + str(i) + str(layers_output[i].size()))
                except:
                    print("go")
                """
                continue
            elif layer_type[i] == 'shortcut':
                skip_step = [int(layer_details[i]['from'])]
            elif layer_type[i] == 'route':
                skip_step = layer_details[i]['layers'].split(",")
            for SS in skip_step:
                SS = int(SS)
                #print("SS" + str(i) + str(SS))
                #print(skip_step)
                
                #print(SS)
                #print(layers_output[i])
                #print(layers_output[i - 1 + SS])
                
                if SS > 0:
                    if layers_output[i] == None:
                        #print(layers_output[SS].size())
                        layers_output[i] = layers_output[SS]
                    else:
                        #print(layers_output[SS].size())
                        layers_output[i] += layers_output[SS]
                else:
                    #print(i + SS)
                    if layers_output[i] == None:
                        #print(layers_output[i + SS].size())
                        layers_output[i] = layers_output[i + SS]
                    else:
                        
                        #print(layers_output[i + SS].size())
                        layers_output[i] += layers_output[i + SS]
            """
            try:
                print("i: " + str(i) + str(layers_output[i].size()))
            except:
                print("go")
            """
        #print(layers_output[138].size())
        return layers_output[161]
    



In [31]:

input = np.array([1 for i in range(608 * 608 * 3 * 1)]).reshape(1, 3, 608, 608)
#target = np.array([0 for i in range(7 * 7 * 30)])
target = np.array([0 for i in range(3 * 19 * 19 * 85)])

input_tensor = torch.Tensor(input)
output_tensor = torch.Tensor(target)

#x = np.array([1 for i in range(608 * 608 * 3)]).reshape(1, 3, 608, 608)
#x = torch.tensor(x)

learning_rate = 0.08
epoch_size = 2
steps_for_printing_out_loss = 1

YOLO_v4_Module_WIP = YOLO_v4_model(layer_details, layer_type)
YOLO_v4_Module_WIP.cuda()



#YOLO_v4_Module_WIP.load_state_dict(torch.load("D:/Installation/yolov4.pt"))

#YOLO_v4_Module_WIP.load_weights("‪D:/Installation/yolov4.weights")
#Model_WIP.to(device)
loss_functioin = nn.MSELoss()
optimizer = optim.SGD(YOLO_v4_Module_WIP.parameters(), lr = learning_rate)


"""

for name, param in YOLO_v4_Module_WIP.named_parameters():
    print('name: ', name)
    print(type(param))
    print('param.shape: ', param.shape)
    print('param.requires_grad: ', param.requires_grad)
    print('=====')
#transfer learning:


for name, param in model.named_parameters():
    if name in ['fc.weight', 'fc.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False
"""

input = input_tensor.cuda()
target = output_tensor.cuda()

def training_model():
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output = YOLO_v4_Module_WIP(input.cuda())
        print(output.size())
        #b_x, b_y, b_w, b_h, objective_p, class_p = YOLO_v4_Module_WIP(input.cuda())
        #output = b_x
        #loss = loss_functioin(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3)))
        
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3), output.size(4)))
        loss.backward()
        optimizer.step()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
    #torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)


training_model()

#YOLO_v4_Module_WIP.state_dict()

[[ 12  16]
 [ 19  36]
 [ 40  28]
 [ 36  75]
 [ 76  55]
 [ 72 146]
 [142 110]
 [192 243]
 [459 401]]
[12, None, None]
[16, None, None]
tensor([[[[0.6661, 0.5000, 0.5728,  ..., 0.6531, 0.5000, 0.5000],
          [0.7666, 0.8081, 0.8431,  ..., 0.5512, 0.9344, 0.9773],
          [0.6206, 0.8418, 0.7212,  ..., 0.5005, 0.5000, 0.7480],
          ...,
          [0.7601, 0.5000, 0.5000,  ..., 0.6086, 0.8753, 0.7279],
          [0.5677, 0.5000, 0.5000,  ..., 0.5742, 0.5000, 0.7921],
          [0.5760, 0.9037, 0.6374,  ..., 0.6776, 0.7177, 0.8499]]]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[[[ 0.6661,  0.5000,  0.5728,  ...,  0.6531,  0.5000,  0.5000],
          [ 1.7666,  1.8081,  1.8431,  ...,  1.5512,  1.9344,  1.9773],
          [ 2.6206,  2.8418,  2.7212,  ...,  2.5005,  2.5000,  2.7480],
          ...,
          [73.7601, 73.5000, 73.5000,  ..., 73.6086, 73.8753, 73.7279],
          [74.5677, 74.5000, 74.5000,  ..., 74.5742, 74.5000, 74.7921],
          [75.5760, 75.903

tensor([[[[ 0.6534,  0.7209,  0.5133,  0.5000,  0.6202,  0.5298,  0.5829,
            0.6005,  0.5000,  0.5000,  0.5000,  0.5876,  0.6035,  0.5298,
            0.5322,  0.6069,  0.5332,  0.6863,  0.6130],
          [ 1.6498,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,
            1.5000,  1.5104,  1.5000,  1.6871,  1.6697,  1.7043,  1.6562,
            1.7172,  1.6337,  1.6377,  1.5000,  1.6651],
          [ 2.7375,  2.5000,  2.6561,  2.5000,  2.5000,  2.7064,  2.6241,
            2.5224,  2.5331,  2.5103,  2.5432,  2.6806,  2.6860,  2.5264,
            2.8012,  2.7137,  2.6199,  2.7338,  2.5000],
          [ 3.7543,  3.7105,  3.6711,  3.5000,  3.5178,  3.5000,  3.6481,
            3.5249,  3.5000,  3.5180,  3.5000,  3.6743,  3.6900,  3.6721,
            3.7194,  3.7379,  3.5588,  3.6064,  3.5000],
          [ 4.6136,  4.6135,  4.5834,  4.5421,  4.5657,  4.5000,  4.5049,
            4.6112,  4.5000,  4.5000,  4.5000,  4.5000,  4.5661,  4.5633,
            4.7116,  4.5864,  4.

tensor([[[[0.5000, 0.6038, 0.6476, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5365, 0.5194, 0.5000, 0.5000, 0.5000, 0.5000, 0.5137,
           0.5000, 0.5902, 0.5970],
          [0.6394, 0.5000, 0.7759, 0.6099, 0.5000, 0.5164, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5700, 0.5337, 0.5126, 0.5000, 0.5000, 0.5972,
           0.5419, 0.7080, 0.6082],
          [0.6102, 0.5000, 0.5000, 0.5000, 0.5000, 0.6135, 0.5125, 0.5855,
           0.5961, 0.6285, 0.5099, 0.5000, 0.5000, 0.5000, 0.7026, 0.6365,
           0.6989, 0.5000, 0.5000],
          [0.6343, 0.5973, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5099,
           0.5000, 0.5553, 0.5283, 0.6755, 0.6568, 0.6109, 0.5146, 0.5107,
           0.7231, 0.5000, 0.5306],
          [0.6820, 0.6091, 0.5534, 0.6082, 0.5130, 0.5000, 0.5021, 0.5818,
           0.5000, 0.5000, 0.5000, 0.6432, 0.5876, 0.5293, 0.6433, 0.6484,
           0.5000, 0.5796, 0.5867],
          [0.6668, 0.5000, 0.6585, 0.6598, 0.5867, 0.5000, 0.5783, 0.5

tensor([[[[ 0.5945,  0.6355,  0.6205,  ...,  0.5675,  0.6156,  0.5939],
          [ 1.7175,  1.6533,  1.5475,  ...,  1.6886,  1.6066,  1.5000],
          [ 2.7162,  2.7313,  2.6413,  ...,  2.6243,  2.6330,  2.5000],
          ...,
          [73.7032, 73.6860, 73.5455,  ..., 73.5000, 73.5415, 73.6124],
          [74.8176, 74.7402, 74.6982,  ..., 74.7113, 74.6374, 74.7645],
          [75.7223, 75.6473, 75.6188,  ..., 75.7719, 75.7190, 75.7711]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
[12, 19, 40]
[16, 36, 28]
tensor([[[[0.5300, 0.5171, 0.5160,  ..., 0.5305, 0.5000, 0.6198],
          [0.5156, 0.5719, 0.5596,  ..., 0.5353, 0.5000, 0.6600],
          [0.5201, 0.6072, 0.5000,  ..., 0.5177, 0.5000, 0.6436],
          ...,
          [0.5497, 0.8363, 0.7066,  ..., 0.9034, 0.7222, 0.9162],
          [0.5679, 0.7836, 0.5610,  ..., 0.7271, 0.5888, 0.8779],
          [0.5000, 0.7049, 0.6825,  ..., 0.5801, 0.7315, 0.9118]]]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[[[

tensor([[[[ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
            0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
            0.5000,  0.5000,  0.5000,  0.5000,  0.5000],
          [ 1.5000,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,
            1.5000,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,  1.5000,
            1.5000,  1.5000,  1.5000,  1.5000,  1.5000],
          [ 2.5000,  2.5000,  2.5000,  2.5000,  2.5000,  2.5000,  2.5000,
            2.5000,  2.5000,  2.5000,  2.5000,  2.5000,  2.5000,  2.5000,
            2.5000,  2.5000,  2.5000,  2.5000,  2.5000],
          [ 3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000,
            3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000,
            3.5000,  3.5000,  3.5000,  3.5000,  3.5000],
          [ 4.5000,  4.5000,  4.5000,  4.5000,  4.5000,  4.5000,  4.5000,
            4.5000,  4.5000,  4.5000,  4.5000,  4.5000,  4.5000,  4.5000,
            4.5000,  4.5000,  4.

tensor([[[[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
           0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5

In [13]:
#all_layerrr

In [14]:
layer_details[160]

{'size': '1',
 'stride': '1',
 'pad': '1',
 'filters': '255',
 'activation': 'linear'}

In [15]:
#all_layers
n = [76, 38, 19]
m = [608 / i for i in n for m in range(6)] 
print(m)


anchor = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
[anchor[i] / m[i] for i in range(len(m))]

[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0]


[1.5,
 2.0,
 2.375,
 4.5,
 5.0,
 3.5,
 2.25,
 4.6875,
 4.75,
 3.4375,
 4.5,
 9.125,
 4.4375,
 3.4375,
 6.0,
 7.59375,
 14.34375,
 12.53125]

In [16]:
class YOLO_v4_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.YOLO_v4_layers = nn.ModuleList()
        
            self.YOLO_v4_layers.append(Conv_Layer_box(in_channel[i], out_channel[i], kernel_size= kernel_size[i], stride = stride[i], activation_func = activation_func[i], batch_normalization = batch_normalization[i]))
    

IndentationError: unexpected indent (<ipython-input-16-13f47200f125>, line 6)

In [ ]:
class abc():
    def __init__(self, qwe, out):
        print(qwe)
        
abc(4,5)

In [ ]:
abc = "1,2,3"
m = abc.split(",")
m
abc = "1"
m = abc.split(",")
m
k = 4

HX = []

for r in k:
    print(r)
    print(k[r])

In [ ]:
weightfile = "D:/Installation/yolov4.weights"
fp = open(weightfile, 'rb')
header = np.fromfile(fp, count=5, dtype=np.int32)
header = torch.from_numpy(header)
seen = header[3]
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

start = 0
ind = -2
buf


In [46]:
HX_weight = YOLO_v4_Module_WIP.state_dict()
i = 0
HX = []

for kk in HX_weight:
    i += 1
    print(kk)
    print(HX_weight[kk].size())
    HX.append(HX_weight[kk].size())
print(i)

all_layers.0.conv_box.0.weight
torch.Size([32, 3, 3, 3])
all_layers.0.conv_box.1.weight
torch.Size([32])
all_layers.0.conv_box.1.bias
torch.Size([32])
all_layers.0.conv_box.1.running_mean
torch.Size([32])
all_layers.0.conv_box.1.running_var
torch.Size([32])
all_layers.0.conv_box.1.num_batches_tracked
torch.Size([])
all_layers.1.conv_box.0.weight
torch.Size([64, 32, 3, 3])
all_layers.1.conv_box.1.weight
torch.Size([64])
all_layers.1.conv_box.1.bias
torch.Size([64])
all_layers.1.conv_box.1.running_mean
torch.Size([64])
all_layers.1.conv_box.1.running_var
torch.Size([64])
all_layers.1.conv_box.1.num_batches_tracked
torch.Size([])
all_layers.2.conv_box.0.weight
torch.Size([64, 64, 1, 1])
all_layers.2.conv_box.1.weight
torch.Size([64])
all_layers.2.conv_box.1.bias
torch.Size([64])
all_layers.2.conv_box.1.running_mean
torch.Size([64])
all_layers.2.conv_box.1.running_var
torch.Size([64])
all_layers.2.conv_box.1.num_batches_tracked
torch.Size([])
all_layers.4.conv_box.0.weight
torch.Size([64, 

In [30]:
d = torch.load("D:/Installation/yolov4.pt")
#d

In [45]:
mm = d['model']
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk].size())
    yolo_v4_size.append(mm[kk].size())
print(i)

module_list.0.Conv2d.weight
torch.Size([32, 3, 3, 3])
module_list.0.BatchNorm2d.weight
torch.Size([32])
module_list.0.BatchNorm2d.bias
torch.Size([32])
module_list.0.BatchNorm2d.running_mean
torch.Size([32])
module_list.0.BatchNorm2d.running_var
torch.Size([32])
module_list.0.BatchNorm2d.num_batches_tracked
torch.Size([])
module_list.1.Conv2d.weight
torch.Size([64, 32, 3, 3])
module_list.1.BatchNorm2d.weight
torch.Size([64])
module_list.1.BatchNorm2d.bias
torch.Size([64])
module_list.1.BatchNorm2d.running_mean
torch.Size([64])
module_list.1.BatchNorm2d.running_var
torch.Size([64])
module_list.1.BatchNorm2d.num_batches_tracked
torch.Size([])
module_list.2.Conv2d.weight
torch.Size([64, 64, 1, 1])
module_list.2.BatchNorm2d.weight
torch.Size([64])
module_list.2.BatchNorm2d.bias
torch.Size([64])
module_list.2.BatchNorm2d.running_mean
torch.Size([64])
module_list.2.BatchNorm2d.running_var
torch.Size([64])
module_list.2.BatchNorm2d.num_batches_tracked
torch.Size([])
module_list.4.Conv2d.weigh

In [44]:
for i in range(len(yolo_v4_size)):
    #print(yolo_v4_size[i] == HX[i])
    if (yolo_v4_size[i] == HX[i]) == False:
        print(yolo_v4_size[i])
        print(HX[i])
        print(i)
        



torch.Size([64, 128, 1, 1])
torch.Size([64, 64, 1, 1])
42
torch.Size([128, 128, 1, 1])
torch.Size([128, 64, 1, 1])
96
torch.Size([256, 256, 1, 1])
torch.Size([256, 128, 1, 1])
222
torch.Size([512, 512, 1, 1])
torch.Size([512, 256, 1, 1])
348
torch.Size([1024, 1024, 1, 1])
torch.Size([1024, 512, 1, 1])
426
torch.Size([512, 2048, 1, 1])
torch.Size([512, 512, 1, 1])
450
torch.Size([256, 512, 1, 1])
torch.Size([256, 256, 1, 1])
480
torch.Size([128, 256, 1, 1])
torch.Size([128, 128, 1, 1])
522
torch.Size([256, 512, 1, 1])
torch.Size([256, 256, 1, 1])
566
torch.Size([512, 1024, 1, 1])
torch.Size([512, 512, 1, 1])
610
